<a href="https://colab.research.google.com/github/Dunckleosteus/Peroskite/blob/unstable/Perovskite_Supervised_Classification_FilteredData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Scope> Predict crystal structure for a given set of perovskite crystal with specified properties


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
raw_data = pd.read_csv('Crystal_structure.csv')

raw_data.head(5)

In [ ]:
raw_data.columns = raw_data.columns.str.replace(' ', '_')
raw_data.head(5)

In [ ]:
raw_data.describe()

In [ ]:
raw_data.dtypes

In [ ]:
import pandas as pd

# Reload the data if raw_data is not a DataFrame
#raw_data = pd.read_csv('/content/drive/MyDrive/ML_Perovskite/Crystal_structure.csv')
raw_data.columns = raw_data.columns.str.replace(' ', '_')

# Now create the copy
data_copy = raw_data.copy()

# Print the first 5 rows
data_copy.head(5)

In [ ]:
# @title Lowest_distortion

from matplotlib import pyplot as plt
import seaborn as sns
data_copy.groupby('Lowest_distortion').size().plot(kind='bar', color=sns.palettes.mpl_palette('Blues_d'))
plt.gca().spines[['top', 'right',]].set_visible(False)
plt.xlabel("Lowest Distortion Category")  # Add x-axis label
plt.ylabel("Frequency")
plt.xticks(rotation=0, size=8)
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Convert 'τ' to numeric, handling errors
data_copy['τ'] = pd.to_numeric(data_copy['τ'], errors='coerce')

# 2. Remove rows with NaN values (if any) resulting from the conversion
data_copy = data_copy.dropna(subset=['τ'])

# 3. Create the histogram
plt.figure(figsize=(8, 6))
sns.histplot(data_copy['τ'], bins=20, kde=True)
plt.xlabel("τ Values")
plt.ylabel("Frequency")
plt.title("Distribution of τ")
plt.show()

In [ ]:
data_copy.dtypes
value_counts_per_column = data_copy.count()
print(value_counts_per_column)

In [ ]:
data_copy.shape

Removed A,B, In literature columns because they did not bring any significant information and the τ column because it had numeorous missing values


In [ ]:
remove_columns = ['A', 'B', 'In_literature', 'τ']
new_data = data_copy.copy()
new_data.drop(columns=remove_columns, inplace=True)
new_data.head(5)
new_data.shape

In [ ]:
print(new_data[['v(A)', 'v(B)']].dtypes)

In [ ]:
# Convert 'v(A)' and 'v(B)' to a suitable numeric type
new_data['v(A)'] = pd.to_numeric(new_data['v(A)'], errors='coerce')
new_data['v(B)'] = pd.to_numeric(new_data['v(B)'], errors='coerce')

# Now replace 0 with NaN and drop rows with NaN
new_data['v(A)'] = new_data['v(A)'].replace(0, np.nan)
new_data['v(B)'] = new_data['v(B)'].replace(0, np.nan)
new_data.dropna(inplace=True)
new_data.head(10)


In [ ]:

# Convert 'v(A)' and 'v(B)' to category
new_data['v(A)'] = new_data['v(A)'].astype('category')
new_data['v(B)'] = new_data['v(B)'].astype('category')
new_data['Lowest_distortion']=new_data['Lowest_distortion'].str.replace('-','')

new_data.info()
new_data.shape

In [ ]:
# Create a copy of filtered_data
filtered_data_encoded = new_data.copy()
# Perform one-hot encoding on the copy
filtered_data_encoded = pd.get_dummies(filtered_data_encoded, columns=['v(A)', 'v(B)'], prefix=['v(A)', 'v(B)'])
filtered_data_encoded.head(10)

In [ ]:
# Assuming 'filtered_data_encoded' is your DataFrame with one-hot encoded columns

# List of one-hot encoded columns
one_hot_columns = ['v(A)_1.0', 'v(A)_2.0', 'v(A)_3.0', 'v(A)_4.0', 'v(A)_5.0', 'v(B)_1.0', 'v(B)_2.0', 'v(B)_3.0', 'v(B)_4.0', 'v(B)_5.0'] # Replace with your actual column names if different


# Convert True/False to 1/0 and change data type to float64
for column in one_hot_columns:
    filtered_data_encoded[column] = filtered_data_encoded[column].astype(int).astype('float64')

filtered_data_encoded.head(5)

In [ ]:
filtered_data_encoded['Lowest_distortion'] = filtered_data_encoded['Lowest_distortion'].astype('category')
filtered_data_encoded.dtypes

In [ ]:
# Convert 'l(A-O)(Å)' and 'l(B-O)(Å)' to numeric type
filtered_data_encoded['l(A-O)(Å)'] = pd.to_numeric(filtered_data_encoded['l(A-O)(Å)'], errors='coerce')
filtered_data_encoded['l(B-O)(Å)'] = pd.to_numeric(filtered_data_encoded['l(B-O)(Å)'], errors='coerce')


In [ ]:
#Remove the '-' category
# Replace empty strings with NaN and drop NaN values
filtered_data_encoded['Lowest_distortion'] = filtered_data_encoded['Lowest_distortion'].replace('', np.nan)
filtered_data_encoded.dropna(subset=['Lowest_distortion'], inplace=True)


# Verify removal of '' category
unique_distortions = filtered_data_encoded['Lowest_distortion'].unique()
print("Unique values after removal of '' category:", unique_distortions)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


# Select all numerical columns, including one-hot encoded ones
numerical_data = filtered_data_encoded.select_dtypes(include=['number', 'uint8']) # Include 'uint8' to capture one-hot encoded columns

# Calculate correlation matrix
correlation_matrix = numerical_data.corr()

# Plotting the heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='YlOrRd', fmt=".2f")
plt.title('Correlation Matrix')
plt.show()


# KNN


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# 1. Separate features (X) and target (y)
X = filtered_data_encoded.drop(columns=['Compound', 'Lowest_distortion'])
y = filtered_data_encoded['Lowest_distortion']

# 2. Perform one-hot encoding on the target variable 'Lowest_distortion'
y_encoded = pd.get_dummies(y, prefix=['Lowest_distortion'])

# 3. Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, stratify=y_encoded, random_state=42)

# 4. Define hyperparameter grid for KNN
knn_param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],  # Try different values for k
    'metric': ['euclidean', 'manhattan']  # Try different distance metrics
}

# 5. Create and train the KNN model with GridSearchCV
knn_model = KNeighborsClassifier()
grid_search_knn = GridSearchCV(knn_model, knn_param_grid, cv=5, scoring='accuracy')
grid_search_knn.fit(X_train, y_train)

# 6. Get the best KNN model and its hyperparameters
best_knn_model = grid_search_knn.best_estimator_
best_knn_params = grid_search_knn.best_params_

# 7. Make predictions on the test set
y_pred_knn = best_knn_model.predict(X_test)

# 8. Evaluate the model
accuracy_knn = accuracy_score(y_test, y_pred_knn)
print(f"Best Hyperparameters for KNN: {best_knn_params}")
print(f"Accuracy with Best KNN Model: {accuracy_knn}")

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report


# 9. Calculate and print classification report
report = classification_report(y_test, y_pred_knn, target_names=y_encoded.columns)  # Use target_names for category labels
print(report)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import confusion_matrix

# Assuming y_test and y_pred_knn are from your KNN model training
# and y_encoded contains the one-hot encoded target labels

# 1. No need to convert y_pred_knn as it's already in class labels
#y_pred_knn_labels = np.argmax(y_pred_knn, axis=1)  # This line is causing the error
#target_names = sorted(filtered_data_encoded['Lowest_distortion'].unique())
#y_pred_knn_labels = [target_names[i] for i in y_pred_knn_labels]

# Get unique class labels directly from y_test
target_names = sorted(y_test.unique())

# 2. Create confusion matrix
cm_knn = confusion_matrix(y_test, y_pred_knn, labels=target_names)

# 3. Plotting the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm_knn, annot=True, fmt="d", cmap="Blues",
            xticklabels=target_names, yticklabels=target_names)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix for KNN Model (One-Hot Encoded)")
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve

# Assuming you have X_train, y_train, and best_knn_model from your previous code

# Create CV training and test scores for various training set sizes
train_sizes, train_scores, test_scores = learning_curve(
    best_knn_model, X_train, y_train, cv=5, scoring='accuracy',
    n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5) # Adjust train_sizes as needed
)

# Calculate mean and standard deviation for training set scores
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)

# Calculate mean and standard deviation for test set scores
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

# Plot the learning curve
plt.figure(figsize=(10, 6))
plt.title("Learning Curve for KNN Model")
plt.xlabel("Training Examples")
plt.ylabel("Score")
plt.grid()

plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.1,
                 color="r")
plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.1, color="g")
plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
         label="Training score")
plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
         label="Cross-validation score")

plt.legend(loc="best")
plt.show()

#Decision Tree

In [ ]:

import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report



In [ ]:
# 1. Separate features (X) and target (y)
X = filtered_data_encoded.drop(columns=['Compound', 'Lowest_distortion'])  #dropped columns that are not useful or are highly correlated
y = filtered_data_encoded['Lowest_distortion']



In [ ]:
# 2. Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, stratify=y, random_state=42)  # Adjust test_size and random_state as needed can modify random state to 123

In [ ]:
# 3. Define the hyperparameter grid to search
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [ ]:
# 4. Create and train the Decision Tree model with GridSearchCV
tree_model = DecisionTreeClassifier(random_state=42)
grid_search = GridSearchCV(tree_model, param_grid, cv=3, scoring='accuracy')  # cv=3/5/10
grid_search.fit(X_train, y_train)


In [ ]:
# 5. Get the best hyperparameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_
print(best_params)

In [ ]:
# 6. Make predictions on the test set using the best model
y_pred_tree = best_model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score # Import necessary functions

# Assuming y_pred is already calculated
accuracy = accuracy_score(y_test, y_pred_tree)  # Compare y_test (true values) with y_pred (predicted values)
precision = precision_score(y_test, y_pred_tree, average='weighted')
recall = recall_score(y_test, y_pred_tree, average='weighted')
f1 = f1_score(y_test, y_pred_tree, average='weighted')

# 3. Print the metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

In [ ]:
# 7. Evaluate the model
accuracy = accuracy_score(y_test, y_pred_tree)
print(f"Best Hyperparameters: {best_params}")
print(f"Accuracy with Best Model: {accuracy}")

In [ ]:
# Get the unique class labels from the original 'Lowest_distortion' column in filtered_data_encoded
target_names = sorted(filtered_data_encoded['Lowest_distortion'].unique())

# Generate the classification report
report = classification_report(y_test, y_pred_tree, target_names=target_names)
print(report)

In [ ]:
#further analysis
#Further analysis and visualization can be done using the following libraries
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 8))
plot_tree(best_model, feature_names=X.columns, class_names=['Cubic', 'Orthorhombic', 'Rhombohedral', 'Tetragonal'], filled=True)
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming you have y_test (true labels) and y_pred (predicted labels)

# Get unique class labels directly from y_test
target_names = sorted(y_test.unique())

# 2. Create confusion matrix
cm_knn = confusion_matrix(y_test, y_pred_tree, labels=target_names)

# 3. Plotting the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm_knn, annot=True, fmt="d", cmap="Blues",
            xticklabels=target_names, yticklabels=target_names)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix for Decision Tree (One-Hot Encoded)")
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve

# Assuming you have X_train, y_train, and best_model (Decision Tree) from your previous code

# Create CV training and test scores for various training set sizes
train_sizes, train_scores, test_scores = learning_curve(
    best_model, X_train, y_train, cv=5, scoring='accuracy',
    n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5) # Adjust train_sizes as needed
)

# Calculate mean and standard deviation for training set scores
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)

# Calculate mean and standard deviation for test set scores
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

# Plot the learning curve
plt.figure(figsize=(10, 6))
plt.title("Learning Curve for Decision Tree Model")
plt.xlabel("Training Examples")
plt.ylabel("Score")
plt.grid()

plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.1,
                 color="r")
plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.1, color="g")
plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
         label="Training score")
plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
         label="Cross-validation score")

plt.legend(loc="best")
plt.show()

#RANDOM FOREST

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Separate features (X) and target (y)
X = filtered_data_encoded.drop(columns=['Compound', 'Lowest_distortion'])
y = filtered_data_encoded['Lowest_distortion']

# 2. Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, stratify=y, random_state=42)

# 3. Define the hyperparameter grid to search
param_grid = {
    'n_estimators': [100, 200, 300],  # Number of trees in the forest
    'max_depth': [None, 5, 10, 15],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4]  # Minimum number of samples required to be at a leaf node
}

# 4. Create and train the Random Forest model with GridSearchCV
rf_model = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# 5. Get the best hyperparameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_
print(f"Best Hyperparameters: {best_params}")

# 6. Make predictions on the test set using the best model
y_pred_rf = best_model.predict(X_test)

# 7. Evaluate the model
accuracy = accuracy_score(y_test, y_pred_rf)
print(f"Accuracy with Best Model: {accuracy}")

# 8. Generate classification report
target_names = sorted(filtered_data_encoded['Lowest_distortion'].unique())
report = classification_report(y_test, y_pred_rf, target_names=target_names)
print(report)

# 9. Create and plot confusion matrix
cm = confusion_matrix(y_test, y_pred_rf, labels=target_names)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=target_names, yticklabels=target_names)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix for Random Forest Model")
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming 'best_model' is your trained Random Forest model
# and 'X' is your DataFrame containing the features

# Get feature importances
feature_importances = best_model.feature_importances_

# Create a DataFrame to store feature importances
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
})

# Sort the DataFrame by importance in descending order
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Plot the feature importances
plt.figure(figsize=(10, 6))
plt.barh(feature_importance_df['Feature'], feature_importance_df['Importance'])
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.title("Feature Importance in Random Forest Model")
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
from sklearn.ensemble import RandomForestClassifier

# Assuming you have X_train, y_train, and best_model from your previous code

# Create CV training and test scores for various training set sizes
train_sizes, train_scores, test_scores = learning_curve(
    best_model, X_train, y_train, cv=5, scoring='accuracy',
    n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5)  # Adjust train_sizes as needed
)

# Calculate mean and standard deviation for training set scores
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)

# Calculate mean and standard deviation for test set scores
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

# Plot the learning curve
plt.figure(figsize=(10, 6))
plt.title("Learning Curve for Random Forest Model")
plt.xlabel("Training Examples")
plt.ylabel("Score")
plt.grid()

plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.1,
                 color="r")
plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.1, color="g")
plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
         label="Training score")
plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
         label="Cross-validation score")

plt.legend(loc="best")
plt.show()


Running with less estimators




In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

X, y = filtered_data_encoded[["tG", "ΔENR", 'EN(B)']], filtered_data_encoded["Lowest_distortion"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

param_grid = {
    "n_estimators":[100, 200, 300],
    "max_depth":[None, 5, 10, 15],
    "min_samples_split":[2, 5, 10],
    "min_samples_leaf":[1, 2, 4]
}
rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring="accuracy")
grid_search.fit(X_train, y_train)
# performance of model
best_params = grid_search.best_params_
best_model  = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy with Best Model: {accuracy}")


In [ ]:
# generate classification report
target_names = sorted(filtered_data_encoded["Lowest_distortion"].unique())
report = classification_report(y_test, y_pred, target_names = target_names)
print(report)


In [ ]:
cm = confusion_matrix(y_test, y_pred, labels=target_names)  # Create confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=target_names)  # Create display object
disp.plot(cmap='Blues')  # Plot the confusion matrix with a blue colormap
plt.title("Confusion Matrix for Random Forest Model")  # Add a title
plt.show()  # Display the plot

# Predicting the missing lowest_distortion categories


In [ ]:
print(len(data_copy.loc[data_copy["Lowest_distortion"] == '-', :]))


In [ ]:
df_subset = data_copy[data_copy['Lowest_distortion'] == '-'].copy()

In [ ]:
df_subset.shape

There are 53 points in the dataset with no lowest_distortion category that we will try to predict with the random forest, which had the best accuracy


In [ ]:
df_subset.dtypes

In [ ]:
# Assuming df_subset is already defined

# 1. Create a copy to avoid modifying the original dataframe
df_subset_encoded = df_subset.copy()

# 2. Perform one-hot encoding on 'v(A)' and 'v(B)' columns
df_subset_encoded = pd.get_dummies(df_subset_encoded, columns=['v(A)', 'v(B)'], prefix=['v(A)', 'v(B)'])

# 3. View the first few rows to check the encoding
df_subset_encoded.head()

In [ ]:
remove_columns = ['A', 'B', 'In_literature', 'τ']
df_subset_encoded.drop(columns=remove_columns, inplace=True)
df_subset_encoded.head(5)
df_subset_encoded.shape

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Separate features (X) and target (y)
X = filtered_data_encoded.drop(columns=['Compound', 'Lowest_distortion'])
y = filtered_data_encoded['Lowest_distortion']

# 2. Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, stratify=y, random_state=42)

# 3. Define the hyperparameter grid to search
param_grid = {
    'n_estimators': [100, 200, 300],  # Number of trees in the forest
    'max_depth': [None, 5, 10, 15],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4]  # Minimum number of samples required to be at a leaf node
}

# 4. Create and train the Random Forest model with GridSearchCV
rf_model = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# 5. Get the best hyperparameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_
print(f"Best Hyperparameters: {best_params}")

# Ensure df_subset_encoded has the same columns as X_train
df_subset_encoded = df_subset_encoded.reindex(columns=X_train.columns, fill_value=0)

# 8. Make predictions using the trained model on the subset:
y_pred = best_model.predict(df_subset_encoded)


# 9. Add the predicted distortions back to the original subset:
df_subset_encoded['Predicted_Lowest_distortion'] = y_pred



In [ ]:
print(df_subset_encoded['Predicted_Lowest_distortion'])

In [ ]:
# Generate histogram
plt.figure(figsize=(8, 6))  # Adjust figure size if needed
plt.hist(df_subset_encoded['Predicted_Lowest_distortion'], bins=df_subset_encoded['Predicted_Lowest_distortion'].nunique(), edgecolor='black')
plt.xlabel("Predicted Lowest Distortion")
plt.ylabel("Frequency")
plt.title("Distribution of Predicted Lowest Distortion")
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.tight_layout()  # Adjust layout for better spacing
plt.show()

In [ ]:
models = ['KNN', 'Decision Tree', 'Random Forest']
accuracies = [accuracy_score(y_test, y_pred_knn),
              accuracy_score(y_test, y_pred_tree),
              accuracy_score(y_test, y_pred_rf)]

# Create bar chart
plt.figure(figsize=(8, 6))
plt.bar(models, accuracies, color=['blue', 'green', 'red'])
plt.xlabel("Model")
plt.ylabel("Accuracy")
plt.title("Comparison of Model Accuracies")
plt.ylim(0, 1)  # Set y-axis limits to 0-1 for accuracy
plt.show()